In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9MoGKNjyxriFivoPOl2m")
project = rf.workspace("signaturedetection-im4xu").project("leaf_detection-fcyc2")
version = project.version(1)
dataset = version.download("yolov12")
                

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 783.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
  Using cached matplotlib-3.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 3.3 MB/s eta 0:00:0000:0100:01
  Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (762 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached contourpy-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (326 kB)
  Using cached fonttools-4


Extracting Dataset Version Zip to leaf_detection-1 in yolov12:: 100%|██████████| 162/162 [00:00<00:00, 17780.38it/s]


In [8]:
import os
import cv2
import shutil
import albumentations as A

# Define paths
dataset_path = "/home/sourav/workplace/leaf_disease_detection/leaf_detection-1"  # Update this with your dataset path
output_path = "/home/sourav/workplace/leaf_disease_detection/cropped_data"  # Update this with your desired output path
dataset_types = ["train", "valid", "test"]

# Define augmentation pipelines
augmentations = [
    A.Compose([
        A.HorizontalFlip(p=1.0),
        A.RandomBrightnessContrast(p=1.0),
        A.Rotate(limit=30, p=1.0),
        A.GaussNoise(p=1.0)
    ]),
    A.Compose([
        A.VerticalFlip(p=1.0),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=1.0),
        A.Blur(blur_limit=3, p=1.0)
    ]),
    A.Compose([
        A.CLAHE(p=1.0),
        A.ColorJitter(p=1.0),
        A.RandomGamma(p=1.0)
    ])
]

# Function to create class-wise directories
def create_class_dirs(base_path, class_ids):
    for dataset_type in dataset_types:
        for class_id in class_ids:
            class_dir = os.path.join(base_path, dataset_type, str(class_id))
            os.makedirs(class_dir, exist_ok=True)

# Function to extract and augment objects
def extract_objects():
    class_ids = set()
    
    for dataset_type in dataset_types:
        images_path = os.path.join(dataset_path, dataset_type, "images")
        labels_path = os.path.join(dataset_path, dataset_type, "labels")
        
        for label_file in os.listdir(labels_path):
            if not label_file.endswith(".txt"):
                continue
            
            image_file = label_file.replace(".txt", ".jpg")  # Change to .png if needed
            image_path = os.path.join(images_path, image_file)
            label_path = os.path.join(labels_path, label_file)
            
            if not os.path.exists(image_path):
                continue
            
            image = cv2.imread(image_path)
            h, w, _ = image.shape
            
            with open(label_path, "r") as file:
                lines = file.readlines()
                
                for idx, line in enumerate(lines):
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    class_ids.add(class_id)
                    x_center, y_center, box_width, box_height = map(float, parts[1:])
                    
                    # Convert YOLO format to pixel values
                    x_min = int((x_center - box_width / 2) * w)
                    y_min = int((y_center - box_height / 2) * h)
                    x_max = int((x_center + box_width / 2) * w)
                    y_max = int((y_center + box_height / 2) * h)
                    
                    cropped_object = image[y_min:y_max, x_min:x_max]
                    
                    if cropped_object.size == 0:
                        continue
                    
                    output_dir = os.path.join(output_path, dataset_type, str(class_id))
                    os.makedirs(output_dir, exist_ok=True)
                    
                    output_filename = f"{image_file.split('.')[0]}_{idx}.jpg"
                    output_filepath = os.path.join(output_dir, output_filename)
                    cv2.imwrite(output_filepath, cropped_object)
                    
                    # Apply multiple augmentations only for class_id 0
                    if class_id == 0:
                        for aug_idx, aug in enumerate(augmentations):
                            augmented = aug(image=cropped_object)
                            augmented_image = augmented["image"]
                            
                            aug_output_filename = f"{image_file.split('.')[0]}_{idx}_aug{aug_idx}.jpg"
                            aug_output_filepath = os.path.join(output_dir, aug_output_filename)
                            cv2.imwrite(aug_output_filepath, augmented_image)
                    
    print("Cropping and augmentation completed!")
    create_class_dirs(output_path, class_ids)

# Run extraction and augmentation
extract_objects()


/home/sourav/workplace/leaf_disease_detection/leaf_env/lib/python3.11/site-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Cropping and augmentation completed!


In [10]:
len(os.listdir('/home/sourav/workplace/leaf_disease_detection/cropped_data/train/0'))

488